In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm.auto import tqdm

import sys
sys.path.append('../')
from irt import *

device ='cuda:1'
train_data_path = "../../data/train/"
eval_data_path = "../../data/eval/"
irt_models_path = "../../irt_models/"

In [2]:
data_eval = pd.read_csv(eval_data_path+"eval_df.csv")
E_test = torch.load(eval_data_path+"eval_ie.pth", weights_only=True)
data_train = pd.read_csv(train_data_path+"train_val_df.csv")
E_train = torch.load(train_data_path+"train_val_ie.pth", weights_only=True)

Y_test = (torch.tensor(data_eval.score)>=4).float()
Y_train = (torch.tensor(data_train.score)>=4).float()
U_test = torch.tensor(data_eval.worker_id)
U_train = torch.tensor(data_train.worker_id)

FileNotFoundError: [Errno 2] No such file or directory: '../../data/eval/eval_df.csv'

Train with interaction term

In [ ]:
irt_model = IRT(interaction=True)

In [ ]:
irt_model.fit(U_train, E_train, Y_train, train_device=device)
irt_model.save(irt_models_path+'irt_model_interaction')

Loading and scoring

In [10]:
irt_model.load(irt_models_path+'irt_model_interaction')
irt_model.score(U_test, E_test)

{0: {'avg': 0.19132131338119507, 'std_error': 0.024098885551241553},
 1: {'avg': 0.5420615077018738, 'std_error': 0.0753648023310445},
 2: {'avg': 0.2914014160633087, 'std_error': 0.05217672585284246},
 3: {'avg': 0.1966433823108673, 'std_error': 0.07468234016156487},
 4: {'avg': 0.5769295692443848, 'std_error': 0.09351094850149366},
 5: {'avg': 0.6566412448883057, 'std_error': 0.11114708582560222},
 6: {'avg': 0.40760424733161926, 'std_error': 0.024131641188271883},
 7: {'avg': 0.4750557839870453, 'std_error': 0.05928288923880602},
 8: {'avg': 0.3836439251899719, 'std_error': 0.045963553199467766},
 9: {'avg': 0.08000698685646057, 'std_error': 0.026913540437817574},
 10: {'avg': 0.43402421474456787, 'std_error': 0.05856913116169853},
 11: {'avg': 0.20833568274974823, 'std_error': 0.056815544702944844},
 12: {'avg': 0.6462475657463074, 'std_error': 0.09996035151444832},
 13: {'avg': 0.14943020045757294, 'std_error': 0.03550688004825168},
 14: {'avg': 0.5018437504768372, 'std_error': 0.

Train without interaction term

In [3]:
irt_model2 = IRT(interaction=False)

In [ ]:
irt_model2.fit(U_train, E_train, Y_train, ds=[1], train_device=device)
irt_model2.save(irt_models_path+'irt_model_no-interaction')

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Loading and scoring

In [ ]:
irt_model2.load(irt_models_path+'irt_model_no-interaction')
irt_model2.score(U_test, E_test)

Some plots

probs = forward(Theta1.cpu(), Theta2.cpu(), U_test, E_test, Ma.cpu(), Mb.cpu()).detach().numpy()
y = Y_test.detach().numpy()

from matplotlib.colors import LogNorm

aucs = []
ns = []
ns_train = []
for u in np.unique(U_test.numpy()):
    ind = U_test.numpy()==u
    if np.sum(y[ind])!=0 and np.sum(1-y[ind])!=0:
        fpr, tpr, thresholds = roc_curve(y[ind], probs[ind])
        aucs.append(float(auc(fpr, tpr)))
        ns.append(int(np.sum(ind)))
        ns_train.append(int(np.sum(U_train.cpu().numpy()==u)))

sc = plt.scatter(ns, aucs, c=ns_train, cmap='viridis', norm=LogNorm(), edgecolor='k')
plt.xscale('log')
plt.colorbar(sc, label='Train size (ns_train, log scale)')  # Add colorbar in log scale
plt.ylabel('AUC')
plt.xlabel('Test size (ns)')
#plt.title('AUC vs Test Size (Colored by Train Size)')
plt.show()

plt.hist(probs[y==0],alpha=.3,density=True)
plt.hist(probs[y==1],alpha=.3,density=True)
plt.show()

np.mean((probs>.5).astype(int)==y)

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

# Example data
# True binary labels
y_true = y

# Predicted probabilities or scores for the positive class
y_scores = probs

# Calculate false positive rate (fpr), true positive rate (tpr), and thresholds
fpr, tpr, thresholds = roc_curve(y_true, y_scores)

# Calculate AUC
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')  # Diagonal line
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.grid(alpha=0.3)
plt.show()
